# An Introduction to analyzing Formamentis Networks using emoatlas


❗ **If this is the first time you're using EmoAtlas, it is suggest to consult the Starting_guide.ipynb before reading this notebook ([Github link](https://github.com/alfonsosemeraro/emoatlas/blob/main/demos/Starting_guide.ipynb)).** ❗



In this introduction, we will see what functions are offered by emoatlas for the analysis of Formamentis Networks.

It is important to note that this notebook will deal with analysis techniques from a high-level perspective and not a statistical one. Its scope will be limited to showing examples of some analysis techniques for analyzing these TFMN (Textual forma mentis networks). There are many techniques that have been developed for network analysis, so the reader is invited to consult existing literature for further techniques and methods.

If you are interested in a more sophisticated article of what is possible with EmoAtlas, please refer to [Social discourse and reopening after COVID-19: A post-lockdown analysis of flickering emotions and trending stances in Italy](https://firstmonday.org/ojs/index.php/fm/article/download/10881/9987).

---



## Setup

In [2]:
from emoatlas import EmoScores

In [3]:
emos = EmoScores()

sample_text = """In this text, we prefer dogs becase they are happy and dogs are positive.
Good dogs make good friends. Cats however are bad and have negative links. We dislike cats.
And even if some hounds can be aggressive, I do love our canine friends."""

fmnt = emos.formamentis_network(sample_text) 

# We will see that fmnt is a Named Tuple of edges and vertices.
print(fmnt)

FormamentisNetwork(edges=[('prefer', 'they'), ('text', 'we'), ('dog', 'they'), ('happy', 'they'), ('happy', 'prefer'), ('happy', 'we'), ('dog', 'prefer'), ('dog', 'positive'), ('dog', 'we'), ('positive', 'prefer'), ('positive', 'we'), ('dog', 'friend'), ('dog', 'good'), ('cat', 'link'), ('bad', 'link'), ('friend', 'hound'), ('canine', 'friend'), ('dog', 'hound'), ('prefer', 'text'), ('dog', 'text'), ('prefer', 'we'), ('they', 'we'), ('dog', 'happy'), ('friend', 'good'), ('bad', 'cat'), ('link', 'negative'), ('cat', 'dislike'), ('dislike', 'we'), ('cat', 'we'), ('I', 'love'), ('friend', 'love'), ('hound', 'love'), ('aggressive', 'love'), ('canine', 'love'), ('aggressive', 'hound'), ('I', 'hound'), ('I', 'aggressive'), ('I', 'friend'), ('aggressive', 'friend'), ('I', 'canine')], vertices=['canine', 'we', 'good', 'bad', 'link', 'dog', 'friend', 'text', 'dislike', 'prefer', 'cat', 'positive', 'negative', 'aggressive', 'they', 'I', 'happy', 'love', 'hound'])


---

# Utilities

## Getting edgelists
Depending on the type of analysis you want to perform with the networks, it may be useful to have access to the edgelist and vertices. Returning these elements is quite easy: once the fmnt is defined, it will only be necessary to use ```.edges``` or ```.vertices``` .

In [4]:
fmnt = emos.formamentis_network(sample_text) 
# Returns a list of all the edges
print(fmnt.edges[0:5])
# Returns a list of all the vertices.
print(fmnt.vertices[0:5])

[('prefer', 'they'), ('text', 'we'), ('dog', 'they'), ('happy', 'they'), ('happy', 'prefer')]
['canine', 'we', 'good', 'bad', 'link']


If you are interested in representing also non-syntactic relationships, **multiplex must be set to True** and you need to specify the relationships with **semantic_enrichment**.

After doing so, fmnt will be a dictionary of edgelists, as such, you will need to index this dictionary as is done in the example below:

In [5]:
fmnt = emos.formamentis_network(sample_text, semantic_enrichment=['synonyms','hypernyms'],multiplex=True) 
fmnt.edges['synonyms'] #getting the synonyms edgelist.

[('dog', 'hound')]